In [28]:
from openimages.download import download_dataset
import os
import shutil
from sklearn.model_selection import train_test_split
import glob as glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np
import re
import ast
import random
import pandas as pd
from PIL import Image
import albumentations as A
from collections import namedtuple
from albumentations.pytorch.transforms import ToTensorV2
from pathlib import Path

np.random.seed(42)

In [29]:
TRAIN = True
EPOCHS = 10

## Download and Split the Data

In [ ]:
# select classes to be downloaded
class_names = ["Alpaca"]

# download the images and annotations for the specified classes
if __name__ == "__main__":
    download_dataset(dest_dir='data/', class_labels=class_names, annotation_format='darknet')

In [ ]:
# rename directory for yolo format
current_dir = 'data/alpaca/darknet'
new_dir = 'data/alpaca/labels'

os.rename(current_dir, new_dir)

print(f"Directory renamed from {current_dir} to {new_dir}")

In [33]:
# define paths to images and labels
base_dir = 'data/alpaca'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# count the number of files in each directory
num_images = len([name for name in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, name))])
num_labels = len([name for name in os.listdir(labels_dir) if os.path.isfile(os.path.join(labels_dir, name))])

print(f"Number of images: {num_images}")
print(f"Number of labels: {num_labels}")

Number of images: 142
Number of labels: 142


In [34]:
# create subdirectories
subdirs = ['train', 'validation', 'test']
for subdir in subdirs:
    os.makedirs(os.path.join(images_dir, subdir), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, subdir), exist_ok=True)

In [35]:
# set list of files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

# sort the files to ensure matching
image_files.sort()
label_files.sort()

In [36]:
# split the data
train_images, temp_images, train_labels, temp_labels = train_test_split(image_files, label_files, test_size=48, random_state=42)
validation_images, test_images, validation_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=34, random_state=42)

In [37]:
# function to move files to respective directories
def move_files(file_list, src_dir, dest_dir):
    for file in file_list:
        shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

In [38]:
# move the files to new directories
move_files(train_images, images_dir, os.path.join(images_dir, 'train'))
move_files(validation_images, images_dir, os.path.join(images_dir, 'validation'))
move_files(test_images, images_dir, os.path.join(images_dir, 'test'))

move_files(train_labels, labels_dir, os.path.join(labels_dir, 'train'))
move_files(validation_labels, labels_dir, os.path.join(labels_dir, 'validation'))
move_files(test_labels, labels_dir, os.path.join(labels_dir, 'test'))

## Data Exploration

### Helper Functions

In [57]:
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    """
    Convert YOLO bounding box format to [xmin, ymin, width, height] format.
    """
    class_id, x_center, y_center, width, height = map(float, yolo_bbox.split())
    xmin = (x_center - width / 2) * img_width
    ymin = (y_center - height / 2) * img_height
    bbox_width = width * img_width
    bbox_height = height * img_height

    return class_id, xmin, ymin, bbox_width, bbox_height

In [58]:
def visualize_bounding_boxes(image_path, label_path):
    """
    Visualize bounding boxes on the image.
    """
    # Read image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_height, img_width, _ = image.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Read bounding box annotations
    with open(label_path, 'r') as f:
        bboxes = f.readlines()

    # Plot each bounding box
    for bbox in bboxes:
        class_id, xmin, ymin, bbox_width, bbox_height = yolo_to_bbox(bbox, img_width, img_height)
        rect = patches.Rectangle((xmin, ymin), bbox_width, bbox_height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()

In [59]:
def get_first_n_files(directory, n):
    """
    Get the first n files in the directory sorted by their names.
    """
    files = sorted([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
    return files[:n]

In [60]:
def process_files(image_dir, label_dir, n):
    """
    Process the first n files in the directory.
    """
    # get the first n files from the image and label directories
    image_files = get_first_n_files(image_dir, n)
    label_files = get_first_n_files(label_dir, n)

    # import images and labels and visualize
    for image_file, label_file in zip(image_files, label_files):
        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, label_file)
        print(f"Processing {image_path} and {label_path}")
        visualize_bounding_boxes(image_path, label_path)

### Plot some images

In [ ]:
# select directories and number of images
image_dir = 'data/alpaca/images/train'
label_dir = 'data/alpaca/labels/train'
n = 5

# visualize
process_files(image_dir, label_dir, n)

## Data Augmentation

### Helper Functions

In [55]:
def create_bboxes_df(directory):
    """
    Returns dataframe with bounding boxes for each image
    """
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    components = line.strip().split()
                    x_center = float(components[1])
                    y_center = float(components[2])
                    width = float(components[3])
                    height = float(components[4])

                    image_id = os.path.splitext(filename)[0]

                    data.append([image_id, x_center, y_center, width, height])

    return pd.DataFrame(data, columns=['image_id', 'x_center', 'y_center', 'width', 'height'])

In [56]:
def draw_rect(img, bboxes, color=(255, 0, 0)):
    """
    Draws bounding boxes on a given image
    """
    img = img.copy()
    height, width, _ = img.shape

    for bbox in bboxes:
        # convert YOLO format to x_min, y_min, x_max, y_max
        x_center, y_center, box_width, box_height = bbox[0], bbox[1], bbox[2], bbox[3]

        x_min = int((x_center - box_width / 2) * width)
        y_min = int((y_center - box_height / 2) * height)
        x_max = int((x_center + box_width / 2) * width)
        y_max = int((y_center + box_height / 2) * height)

        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, thickness=2)

    return img

def read_img(img_id):
    """
    Reads the image
    """
    train_dir = 'data/alpaca/images/train'
    img_path = f'{train_dir}/{img_id}.jpg'
    img = cv2.imread(str(img_path))
    return img

def read_bboxes(img_id, df):
    """
    Gets all bounding boxes for a given image
    """
    return df.loc[df.image_id == img_id, 'x_center y_center width height'.split()].values

def plot_img(img_id, df, bbox=False):
    """
    Plot image with bounding boxes
    """
    img = read_img(img_id)
    if bbox:
        bboxes = read_bboxes(img_id, df)
        img = draw_rect(img, bboxes)
    plt.imshow(img)

def plot_multiple_img(img_matrix_list, title_list, ncols, nrows=2, main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=nrows, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize=30)
    fig.subplots_adjust(wspace=0.3, hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
        myaxes[i // ncols][i % ncols].grid(False)
        myaxes[i // ncols][i % ncols].set_xticks([])
        myaxes[i // ncols][i % ncols].set_yticks([])

    plt.show()

In [42]:
train_label_directory = 'data/alpaca/labels/train'
train_labels_df = create_bboxes_df(train_label_directory)

In [ ]:
# choose image
img_id = '0e6ba4d54d478f76'

# get image and bounding boxes
chosen_img = read_img(img_id)
bboxes = read_bboxes(img_id, train_labels_df)
bbox_params = {'format': 'yolo', 'label_fields': ['labels']}

# define augmentations
albumentation_list = [
    A.Compose([A.RandomFog(p=1)], bbox_params=bbox_params),
    A.Compose([A.RandomCrop(p=1, height=512, width=512)], bbox_params=bbox_params),
    A.Compose([A.Rotate(p=1, limit=90)], bbox_params=bbox_params),
    A.Compose([A.RGBShift(p=1)], bbox_params=bbox_params),
    A.Compose([A.RandomSnow(p=1)], bbox_params=bbox_params),
    A.Compose([A.VerticalFlip(p=1)], bbox_params=bbox_params)
]

# define augmentation names
titles_list = ['Original',
              'RandomFog',
              'RandomCrop',
              'Rotate,'
              'RGBShift',
              'RandomSnow',
              'VerticalFlip']

# add original and augmented images to the list
img_matrix_list = [draw_rect(chosen_img, bboxes)]
for aug_type in albumentation_list:
    anno = aug_type(image=chosen_img, bboxes=bboxes, labels=np.ones(len(bboxes)))
    img = draw_rect(anno['image'], anno['bboxes'])
    img_matrix_list.append(img)

# plot image and its augmentations
plot_multiple_img(img_matrix_list, titles_list, ncols=3, main_title='Different Types of Augmentations with Bounding Boxes')

In [44]:
def augment_and_save(img_id, df, augmentations, output_img_dir, output_lbl_dir):
    """
    Augment images and labels in a given directories and save them
    """
    chosen_img = read_img(img_id)
    bboxes = read_bboxes(img_id, df)

    # apply augmentations and save results
    for aug_idx, aug in enumerate(augmentations):
        anno = aug(image=chosen_img, bboxes=bboxes, labels=np.ones(len(bboxes)))
        img = anno['image']
        aug_bboxes = anno['bboxes']

        # save the augmented image
        aug_img_name = f'{img_id}_aug{aug_idx}.jpg'
        cv2.imwrite(os.path.join(output_img_dir, aug_img_name), img)

        # save the augmented labels
        aug_lbl_name = f'{img_id}_aug{aug_idx}.txt'
        with open(os.path.join(output_lbl_dir, aug_lbl_name), 'w') as f:
            for bbox in aug_bboxes:
                f.write('0 '+ ' '.join(map(str, bbox)) + '\n')

In [45]:
def get_all_image_ids(directory):
    """
    Get all images from a given directory
    """
    return [os.path.splitext(filename)[0] for filename in os.listdir(directory) if filename.endswith('.jpg')]

In [47]:
# define bbox parameters and augmentations to perform
bbox_params = {'format': 'yolo', 'label_fields': ['labels']}

albumentation_list = [
    A.Compose([A.RandomFog(p=1)], bbox_params=bbox_params),
    A.Compose([A.RandomCrop(p=1, height=512, width=512)], bbox_params=bbox_params),
    A.Compose([A.Rotate(p=1, limit=90)], bbox_params=bbox_params),
    A.Compose([A.RGBShift(p=1)], bbox_params=bbox_params),
    A.Compose([A.RandomSnow(p=1)], bbox_params=bbox_params),
    A.Compose([A.VerticalFlip(p=1)], bbox_params=bbox_params)
]

# define directories for outputting images and labels
output_img_dir = 'data/alpaca/images/train'
output_lbl_dir = 'data/alpaca/labels/train'

image_ids = get_all_image_ids(output_img_dir)

In [48]:
# augment and save images
for img_id in image_ids:
    augment_and_save(img_id, train_labels_df, albumentation_list, output_img_dir, output_lbl_dir)

In [49]:
# define paths to train images and labels
base_dir = 'data/alpaca'
images_dir = os.path.join(base_dir, 'images/train')
labels_dir = os.path.join(base_dir, 'labels/train')

# count the number of files in each directory
num_images = len([name for name in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, name))])
num_labels = len([name for name in os.listdir(labels_dir) if os.path.isfile(os.path.join(labels_dir, name))])

print(f"Number of images after Augmentation: {num_images}")
print(f"Number of labels after Augmentation: {num_labels}")

Number of images after Augmentation: 658
Number of labels after Augmentation: 658


## Model Training

### Helper Functions for Logging and Validation

In [8]:
def set_res_dir():
    """
    Set directory to store the results of the next training
    """
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

In [9]:
def monitor_tensorboard():
    """
    Create Tensorboard
    """
    %load_ext tensorboard
    %tensorboard --logdir runs/train

In [ ]:
def show_val_results(RES_DIR):
    """
    Check predictions of a given model on the validation set
    """
    EXP_PATH = f"runs/train/{RES_DIR}"
    validation_pred_images = glob.glob(f"{EXP_PATH}/*_pred.jpg")
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

In [ ]:
def inference(RES_DIR, data_path):
    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f"Current number of inference detection directories: {infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)
    !python detect.py --weights ../runs/train/{RES_DIR}/weights/best.pt \
    --source {data_path} --name {INFER_DIR}
    return INFER_DIR

In [ ]:
def visualize(INFER_DIR):
    INFER_PATH = f"runs/detect/{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*.jpg")
    print(infer_images)
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

### Clone YOLOv5 Repository

In [10]:
# clone yolov5 repository
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd yolov5/
!pwd

In [ ]:
!pip install -r requirements.txt

### Training using YOLOv5

In [13]:
monitor_tensorboard()

Launching TensorBoard...

In [ ]:
RES_DIR = set_res_dir()
if TRAIN:
    !python train.py --data ../data.yaml --weights yolov5s.pt \
    --img 640 --epochs 1 --batch-size 16 --name RES_DIR

### Check Validation Predictions

In [ ]:
show_val_results(RES_DIR)

### Training Medium Model with Freezed Layers

In [ ]:
if TRAIN:
    !python train.py --data ../data.yaml --weights yolov5m.pt \
    --img 640 --epochs 1 --batch-size 16 \
    --freeze 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15

In [ ]:
show_val_results(RES_DIR)